The notebook load the dataset from the Google drive and does inference

In [1]:
# Install all the required packages (borrowed from openWakeWord's automatic training notebook)
running_on_colab = False
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    from google.colab import drive
    drive.mount('/content/drive')
    running_on_colab = True
    restore_dataset = True
    restore_features = True
else:
    print('Not running on CoLab')
    restore_dataset = False
    restore_features = False


Not running on CoLab


In [2]:
import os

if running_on_colab:
    # restoring previous trained model for finetuning from previously generated features
    trained_model_filename = "/content/drive/MyDrive/ColabNotebooks/VoiceAssistant/microWakeWord/trained_models_20240421_164022.tar"
    !cp {trained_model_filename}  .

    model_filename = os.path.basename(trained_model_filename)
    !tar -xvf {model_filename}

if os.path.exists('audio_preprocessor_int8.tflite') == False:
    !wget https://github.com/tensorflow/tflite-micro/raw/main/tensorflow/lite/micro/examples/micro_speech/models/audio_preprocessor_int8.tflite


In [3]:
# download this just as a validation model to test the actual mic/tflite process
if os.path.exists('./okay_nabu.tflite') == False:
    !wget https://github.com/esphome/micro-wake-word-models/raw/main/models/okay_nabu.tflite

--2024-04-29 17:02:07--  https://github.com/esphome/micro-wake-word-models/raw/main/models/okay_nabu.tflite
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/esphome/micro-wake-word-models/main/models/okay_nabu.tflite [following]
--2024-04-29 17:02:07--  https://raw.githubusercontent.com/esphome/micro-wake-word-models/main/models/okay_nabu.tflite
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 115400 (113K) [application/octet-stream]
Saving to: 'okay_nabu.tflite'

okay_nabu.tflite    100%[===================>] 112.70K  --.-KB/s    in 0.08s   

2024-04-29 17:02:08 (1.39 MB/s) - 'okay_nabu.tflite' s

In [4]:
%pip install -q tflite_micro
%pip install -q asciichartpy
%pip install -q sounddevice

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
#import gradio as gr
import tensorflow as tf
from tflite_micro.python.tflite_micro import runtime
import numpy as np
import scipy.signal

2024-04-29 19:47:32.949697: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-29 19:47:32.949742: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-29 19:47:32.950855: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-29 19:47:32.994386: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# the following swithc uses the tflite preprocessor (same one used in the embedded device) as opposed of using
# the microfrontend s/w based one
tflite_prep = True

import tensorflow as tf
from tensorflow.lite.experimental.microfrontend.python.ops import audio_microfrontend_op as frontend_op
from tflite_micro.python.tflite_micro import runtime

preprocessor_model = runtime.Interpreter.from_file("./audio_preprocessor_int8.tflite")
input_details = preprocessor_model.get_input_details(0)
output_details = preprocessor_model.get_output_details(0)
preprocessor_model.print_allocations()

class VectorSplitter:
    def __init__(self, chunk_size=480):
        self.chunk_size = chunk_size
        self.remainder = np.zeros(160)

    def split_into_chunks(self, vector):
        chunks = []
        #print(f"Splitting vector of size {len(vector)}, stored remainder is of size {len(self.remainder)}")
        vector = np.concatenate((self.remainder, vector))
        #print(f"Concatenated vector of size {len(vector)}")
        i = 0
        while i + self.chunk_size <= len(vector):
        #for i in range(0, len(vector), self.chunk_size):
            chunk = vector[i:i + self.chunk_size]
            chunks.append(chunk)
            #print("append chunk ", chunk.shape, " i:", i)
            #self.remainder = vector[-160:]
            #print("Remainder:", self.remainder.shape)
            i += 320
        #if i < len(vector):
        self.remainder = vector[i:]
        #print("end of vector Remainder:", self.remainder.shape)
        return chunks

splitter = VectorSplitter()
def get_features(input):
    if len(input) != 480:
        raise ValueError("Input must be of size 480")
        return
    preprocessor_model.set_input(input.reshape([1,480]).astype(np.int16), 0)
    preprocessor_model.invoke()
    return preprocessor_model.get_output(0)

# this generates the MEL spectrogram features for a given clip
def generate_features_for_clip(clip):

    if tflite_prep == True:
        chunks =  splitter.split_into_chunks(clip)
        matrix = np.array([get_features(chunk) for chunk in chunks])
        return matrix
    else:
        micro_frontend = frontend_op.audio_microfrontend(
            tf.convert_to_tensor(clip),
            sample_rate=16000,
            window_size=30,
            window_step=20,
            num_channels=40,
            upper_band_limit=7500,
            lower_band_limit=125,
            enable_pcan=True,
            min_signal_remaining=0.05,
            out_scale=1,
            out_type=tf.float32)
        output = tf.multiply(micro_frontend, 0.0390625)
        return output.numpy()

def features_generator(generator):
    for data in generator:
        for clip in data:
            yield generate_features_for_clip(clip)

Number of resource variables the model uses =  0


[RecordingMicroAllocator] Arena allocation total 11800 bytes
[RecordingMicroAllocator] Arena allocation head 3040 bytes
[RecordingMicroAllocator] Arena allocation tail 8760 bytes
[RecordingMicroAllocator] 'TfLiteEvalTensor data' used 1032 bytes with alignment overhead (requested 1032 bytes for 43 allocations)
[RecordingMicroAllocator] 'Persistent TfLiteTensor data' used 224 bytes with alignment overhead (requested 224 bytes for 2 tensors)
[RecordingMicroAllocator] 'Persistent buffer data' used 4988 bytes with alignment overhead (requested 4872 bytes for 19 allocations)
[RecordingMicroAllocator] 'NodeAndRegistration struct' used 1936 bytes with alignment overhead (requested 1936 bytes for 22 NodeAndRegistration structs)


In [4]:

infer_model = tf.lite.Interpreter(model_path="./trained_models/alexha/tflite_stream_state_internal_quant/stream_state_internal_quant.tflite", num_threads=1)
#infer_model = tf.lite.Interpreter(model_path="./okay_nabu.tflite", num_threads=1)
infer_model.resize_tensor_input(0, [1,1,40], strict=True)  # initialize with fixed input size
infer_model.allocate_tensors()
infer_model_input_details = infer_model.get_input_details()
infer_model_output_details = infer_model.get_output_details()
print()
print("Input details:")
print(infer_model_input_details)
print()
print("Output details:")
print(infer_model_output_details)
print()


Input details:
[{'name': 'serving_default_input_audio:0', 'index': 0, 'shape': array([ 1,  1, 40], dtype=int32), 'shape_signature': array([ 1,  1, 40], dtype=int32), 'dtype': <class 'numpy.int8'>, 'quantization': (0.9803921580314636, 3), 'quantization_parameters': {'scales': array([0.98039216], dtype=float32), 'zero_points': array([3], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]

Output details:
[{'name': 'StatefulPartitionedCall:0', 'index': 129, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([1, 1], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.00390625, 0), 'quantization_parameters': {'scales': array([0.00390625], dtype=float32), 'zero_points': array([0], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]



INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [5]:
import sounddevice as sd
import numpy as np
import asciichartpy
from IPython.display import clear_output
# Initialize a list to hold the last 100 data points
last_100_data_points = [0] * 100

# Define your callback here
def process_audio_callback(indata, frames, time, status):
    global last_100_data_points
    indata = (indata.flatten()*32767).astype(np.int16)
    #print("indata shape:", indata.shape)
    res = generate_features_for_clip(indata)
    #print("res shape:", res.shape)
    # Get predictions
    for row in res:
        row1 = row.astype(np.int8)
        row3 = row1.reshape([1,1,40])
        infer_model.set_tensor(infer_model_input_details[0]['index'], row3)
        infer_model.invoke()
        pred = infer_model.get_tensor(infer_model_output_details[0]['index'])
        # Update the list of last 100 data points
        last_100_data_points = last_100_data_points[1:] + [pred[0,0]]
        last_100_data_points[0] = 255

# Set the callback to be called every 500 ms
stream = sd.InputStream(callback=process_audio_callback, channels=1, blocksize=int(320), samplerate = 16000)
with stream:
    while True:
        sd.sleep(500)
        # Clear the console
        clear_output(wait=True)
        #print(last_100_data_points)
        print(asciichartpy.plot(last_100_data_points, {"height": 10}))

PortAudioError: Error querying device -1

Backup model for later continued training